In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as plt
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Point 1: 
Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

First of all, let's load in memory the loans_lenders file and have a short look at it.
As you can see below, the file appears as a table with 2 columns: loan_id and lenders.

In [2]:
loan_lenders = pd.read_csv('data/loans_lenders.csv')

In [3]:
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In this kind of configuration, the loan_id column does not contain any duplicate

In [4]:
len(set(loan_lenders['loan_id'])) == loan_lenders['loan_id'].size

True

However, the lenders column contains, for each row, all the lenders having partecipate to the loan split by comma.
For the needs of this reseach, we need to convert the data structure of the loan_lenders table to be in the "First Normal Form": 

A relation is in first normal form if and only if the domain of each attribute contains only atomic (indivisible) values, and the value of each attribute contains only a single value from that domain. https://en.wikipedia.org/wiki/First_normal_form

Below the normalization function, which accept a generic dataframe, column, the separator inside the values of the underlying column and a boolean value which whether to retain the presplit value as it's own row

In [5]:
def normalize(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [6]:
%%time

norm_loan_lenders = normalize(loan_lenders, 'lenders', ',')

CPU times: user 14.6 s, sys: 689 ms, total: 15.3 s
Wall time: 15.2 s


In [7]:
norm_loan_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


In [8]:
norm_loan_lenders.tail()

,loan_id,lenders
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175
1387431,1206486,amy38101311


As you can see above, now there is a single lender for each row, but the uniqueness of loan_id values is broken: this is fine for our purpose, since we need the lender to be single value for each row.
Let's also rename the column from lenders to lender.

In [9]:
norm_loan_lenders.rename(columns={'lenders': 'lender'}, inplace=True)

In [10]:
norm_loan_lenders.head()

,loan_id,lender
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


### Point 2: 
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

For the accomplishment of this point, we need the loan file to be imported. since it's relatively a big file (2.29GB) let's import only the needed column to save memory.

In [11]:
%%time

loans = pd.read_csv('data/loans.csv')[['loan_id', 'country_code', 'loan_amount', 'num_lenders_total', 'disburse_time', 'planned_expiration_time']]

CPU times: user 20.9 s, sys: 1.75 s, total: 22.7 s
Wall time: 24.8 s


Time to focus on disburse_time and planned_expiration_time columns.
They both are in string column, except for nan values which are threated as float by pandas.
Talking of nan's, there are to many values, especially disburse_time column.

In [12]:
loans[['disburse_time', 'planned_expiration_time']].head()

,disburse_time,planned_expiration_time
0,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000
1,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000
2,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000
3,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000
4,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000


In [13]:
print(type(loans.disburse_time[0]))
print(type(loans.planned_expiration_time[0]))

<class 'str'>
<class 'str'>


In [14]:
print(len(loans.planned_expiration_time[loans.planned_expiration_time.isnull()]),
len(loans.disburse_time[loans.disburse_time.isnull()]))

371834 2813


Let's convert to datetime all not nan values of disburse_time and planned_expiration_time.

In [15]:
formatter = '%Y-%m-%d %H:%M:%S.%f +0000' # str date formate of the two columns

In [16]:
%%time 

loans['disburse_time'] =  pd.to_datetime(loans['disburse_time'], format=formatter)
loans['planned_expiration_time'] =  pd.to_datetime(loans['planned_expiration_time'], format=formatter)

CPU times: user 5.48 s, sys: 3.49 ms, total: 5.48 s
Wall time: 5.48 s


Calculate time delta's with datetime columns it's very simple

In [17]:
%%time

loans['duration'] = loans.planned_expiration_time - loans.disburse_time

CPU times: user 31.5 ms, sys: 3.12 ms, total: 34.6 ms
Wall time: 38.9 ms


In [18]:
loans.duration.head()

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: duration, dtype: timedelta64[ns]

There are 14935 loans which duration is negative: it means that the deadline was not satisfied

In [19]:
loans.duration[loans.duration < pd.Timedelta(0)]

63        -31 days +21:40:03
96        -25 days +06:30:11
177       -25 days +21:00:03
207       -33 days +06:30:08
217       -31 days +17:40:03
                 ...        
1419303   -35 days +21:30:03
1419304   -32 days +17:50:04
1419374   -34 days +21:40:03
1419443   -30 days +06:00:03
1419581   -30 days +16:50:04
Name: duration, Length: 14935, dtype: timedelta64[ns]

### Point 3: 
Find the lenders that have funded at least twice

Let's go back work on loan lenders table, normalized. At this point, we're asked to find the lenders wich have contributed to a loan at least two times.
This operation is performed by grouping by lender column and count how many loans they contributed.
Note: let's first have a look on possible duplicates (loan_id - lender).

In [20]:
duplicated_rows = norm_loan_lenders[norm_loan_lenders.duplicated()]

In [21]:
duplicated_rows.size

1592252

For the sake of this activity, i decided to maintain duplicates since the guideline says: lenders that have fundend at least twice. A lender might have funded twice on the same loan, so this fits my needs.

There are 2010222 lenders which have funded at least twice.

In [22]:
%%time

loans_by_lenders = norm_loan_lenders.groupby('lender')['loan_id'].count().reset_index(name='fund_count')

CPU times: user 9.91 s, sys: 17.4 ms, total: 9.93 s
Wall time: 9.92 s


In [23]:
loans_by_lenders = loans_by_lenders[loans_by_lenders.fund_count > 1]

In [24]:
loans_by_lenders.head(15)

,lender,fund_count
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
6,0154884,4
7,0161130,2
8,0169713,3
9,0185429,2
11,0206338,2


In [25]:
loans_by_lenders.size

2010222

### Point 4: 
For each country, compute how many loans have involved that country as borrowers

### Point 5: 
For each country, compute the overall amount of money borrowed

### Point 6: 
Like the previous point, but expressed as a percentage of the overall amount lent

I decided to group together points 4,5,6 since they work on the same dataset (loans) using the same group by structure.
We now focus on country group level. we're asked to compute the number of loans have involved a country as borrower, the overall amount borrowed and the % of amount borrowed wrt overall amount lent.

The idea is to group by country code first, then apply all the aggregate operators needed (count, sum..).

In [26]:
def borrow_summary(dimensions: list) -> pd.DataFrame:
    grouped = loans.groupby(dimensions)
    borrow_summary = grouped['loan_id'].count().reset_index(name='borrow_count')
    borrow_summary['overall_amount_borrowed'] = grouped['loan_amount'].sum().reset_index()['loan_amount']
    overall_amount_lent = loans['loan_amount'].sum()
    borrow_summary['overall_amount_borrowed/total'] = (borrow_summary['overall_amount_borrowed'] / overall_amount_lent) * 100
    return borrow_summary

In [27]:
borrow_by_country = borrow_summary('country_code')

In [28]:
borrow_by_country.head(15)

,country_code,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
0,AF,2337,1967950.00000,0.16657
1,AL,3075,4307350.00000,0.36459
2,AM,13952,22950475.00000,1.94259
3,AZ,10172,14784625.00000,1.25141
4,BA,608,477250.00000,0.04040
5,BF,3489,4085200.00000,0.34578
6,BG,296,375300.00000,0.03177
7,BI,1727,5233450.00000,0.44297
8,BJ,5946,3865825.00000,0.32721
9,BO,25250,44226725.00000,3.74347


In [29]:
borrow_by_country.describe()

,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
count,95.00000,95.00000,95.00000
mean,14943.13684,12435766.05263,1.05260
std,35149.88448,18263180.36163,1.54584
min,1.00000,5000.00000,0.00042
25%,545.00000,752525.00000,0.06370
50%,4681.00000,4181100.00000,0.35390
75%,15117.00000,16728425.00000,1.41594
max,285336.00000,97984600.00000,8.29368


In [30]:
borrow_by_country.sort_values('overall_amount_borrowed', ascending=False)

,country_code,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
61,PH,285336,97984600.00000,8.29368
59,PE,86000,79437775.00000,6.72382
38,KE,143699,66735975.00000,5.64871
65,PY,24787,53964700.00000,4.56772
40,KH,79701,51613525.00000,4.36871
...,...,...,...,...
86,VI,2,10000.00000,0.00085
88,VU,4,9250.00000,0.00078
84,UY,1,8000.00000,0.00068
12,BW,1,8000.00000,0.00068


We can notice that Philippines, Peru and Kenia are the country which borrowed more money with 8.29, 6.72 and 5.64 % of total money lent.

### Point 7: 
Like the three previous points, but split for each year (with respect to disburse_time)

We can here exploit the same function defined before, but grouping by (country_code, year)

In [31]:
borrow_by_year = borrow_summary(['country_code', loans['disburse_time'].dt.year])

In [32]:
borrow_by_year.head(15)

,country_code,disburse_time,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
0,AF,2007.00000,408,194975.00000,0.01650
1,AF,2008.00000,370,365375.00000,0.03093
2,AF,2009.00000,678,585125.00000,0.04953
3,AF,2010.00000,632,563350.00000,0.04768
4,AF,2011.00000,247,245125.00000,0.02075
5,AF,2015.00000,1,6000.00000,0.00051
6,AF,2016.00000,1,8000.00000,0.00068
7,AL,2012.00000,332,413000.00000,0.03496
8,AL,2013.00000,507,786950.00000,0.06661
9,AL,2014.00000,603,902550.00000,0.07639


In [33]:
borrow_by_year.sort_values(['overall_amount_borrowed'], ascending=False).head(20)

,country_code,disburse_time,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
483,PH,2017.00000,53195,17096900.00000,1.44713
482,PH,2016.00000,48262,16218925.00000,1.37281
481,PH,2015.00000,46021,16083375.00000,1.36134
480,PH,2014.00000,37778,13961450.00000,1.18173
294,KE,2015.00000,23560,10257950.00000,0.86826
296,KE,2017.00000,23812,10182050.00000,0.86184
466,PE,2012.00000,11191,9830525.00000,0.83208
519,PY,2016.00000,3840,9819725.00000,0.83117
295,KE,2016.00000,20512,9629650.00000,0.81508
479,PH,2013.00000,24496,9432425.00000,0.79839


### Point 8: 
For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


This is the first assignment which involves multiple datasets at the same time. In this case, we're interested in compute the overall amount of money lent for each lender. 
Since more than one lender can partecipate to the same loan (as we saw in assignment 1), we need to calculate the base contribution for a specific loan (assumption of the assignment).

Let's calculate it as the the loan amount divided by number of lenders

In [34]:
loans['base_contribution'] = loans.loan_amount / loans.num_lenders_total

We need to operate some clean up activity before perform the join operation between loan lenders and loans datasets.
Let's calculate the sets of loan ids available in loans dataset but not in loan_lenders one and viceversa.

In [35]:
ids = set(loans.loan_id) - set(loan_lenders.loan_id)
ids2 = set(loan_lenders.loan_id) - set(loans.loan_id)


Make survive at the filter only loans whit a number of lenders greater than zero and which ids appear in loan_lenders table.

In [36]:
loans_cleaned = loans[(loans.num_lenders_total > 0) & (~loans.loan_id.isin(ids))]

Make survive at the filter only lenders which loan id appears in loans table

In [37]:
norm_cleaned = norm_loan_lenders[~norm_loan_lenders.loan_id.isin(ids2)]

Time to join. For the sake of this operation, I'll use pandas concat function.
After setting up the loan_id column as index for both the datasets, we perform 
an inner join operation to enrich the normalized loan lenders dataset with additional information on the related loans

In [38]:
%%time

joined = pd.concat([norm_cleaned.set_index('loan_id'), loans_cleaned.set_index('loan_id')], axis=1, join='inner')

CPU times: user 4.31 s, sys: 2.3 s, total: 6.61 s
Wall time: 10.6 s


In [39]:
joined.head()

,lender,country_code,loan_amount,num_lenders_total,disburse_time,planned_expiration_time,duration,base_contribution
loan_id,,,,,,,,
483693,muc888,PH,1225.00000,44,2012-09-20 07:00:00,2012-11-17 04:50:01,57 days 21:50:01,27.84091
483693,sam4326,PH,1225.00000,44,2012-09-20 07:00:00,2012-11-17 04:50:01,57 days 21:50:01,27.84091
483693,camaran3922,PH,1225.00000,44,2012-09-20 07:00:00,2012-11-17 04:50:01,57 days 21:50:01,27.84091
483693,lachheb1865,PH,1225.00000,44,2012-09-20 07:00:00,2012-11-17 04:50:01,57 days 21:50:01,27.84091
483693,rebecca3499,PH,1225.00000,44,2012-09-20 07:00:00,2012-11-17 04:50:01,57 days 21:50:01,27.84091


Having retrieved all the need information, let's perform the task assigned. 
First group by lender column, then sum the base_contribution one.

In [40]:
overall_amount_per_lender = joined.groupby('lender')['base_contribution'].sum().reset_index(name='overall_amount_lent')

In [41]:
overall_amount_per_lender.sort_values('overall_amount_lent', ascending=False)

,lender,overall_amount_lent
451868,gooddogg1,6296800.48566
1293514,trolltech4460,5327795.31327
451289,gmct,5224017.78783
636708,jouko1940,4925805.69278
984907,nms,4049180.94569
...,...,...
1473277,jenny4658,6.57895
1431946,ellen9682,6.25000
1581767,richard9878,6.25000
1600428,sharif6818,5.68182


In [42]:
overall_amount_per_lender.describe()

,overall_amount_lent
count,1639026.00000
mean,637.10046
std,12587.00054
min,5.43478
25%,33.48962
50%,88.17220
75%,289.14772
max,6296800.48566


### Point 9: 
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

For the accomplishment of this point, there's need to load the lenders dataset, which give information about lenders.

In [43]:
lenders = pd.read_csv('data/lenders.csv')

As revealed in the assignment, country code column holds lots of nan's

In [44]:
sum(lenders['country_code'].isna()) ### lot of nan's. we must assume 

1458635

Let's replace the nan's with the distribution of country_code column (of the entries which have a value).
First retrieve the relative frequencies of the unique values present in country code column.

In [45]:
s = lenders.country_code.value_counts(normalize=True)

In [46]:
s.head()

US   0.66433
CA   0.07632
GB   0.04310
AU   0.04166
DE   0.01797
Name: country_code, dtype: float64

Take the subset of records which holds the nan's

In [47]:
missing = lenders['country_code'].isnull()

Replace the nan's of the selected rows whit a random value chosen from the distribution s calculated before

In [48]:
lenders.loc[missing,'country_code'] = np.random.choice(s.index, size=len(lenders[missing]),p=s.values)

In [49]:
len(lenders[lenders.country_code.isnull()])

0

Now that nan's values have been replaced, we can perform the join operation between the overall_amount_per_lender dataset computed before and the new lenders one just imported.
Since we're going to perform a join on names, we should first have a look on data quality of the related columns of both the dataset. 

For this didactic assignment, we'll assume that lender names present in the loan_lenders file are a subset of the lenders one. We'll just strip strings removing empty spaces which could have been created by the split operation at point 1

In [50]:
overall_amount_per_lender.lender = overall_amount_per_lender.lender.str.strip()

In [51]:
%%time

joined = pd.concat([overall_amount_per_lender.set_index('lender'), lenders.set_index('permanent_name')], axis=1, join='inner')

CPU times: user 7.41 s, sys: 14.4 ms, total: 7.42 s
Wall time: 7.42 s


In [52]:
joined.head()

,overall_amount_lent,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
000,1485.30966,Chad Graves,Nashville,TN,US,1202872371,NaN,NaN,40.00000,Vincent,0
00000,1249.94736,Nanda,NaN,NaN,DE,1261131466,NaN,"Well, because I can ;o)\r\nThis world belongs ...",39.00000,NaN,1
0002,2201.18046,Lynn & Evan,Apple Valley,CA,US,1192667581,"Retired, Postmaster",To help those that want to help themselves.,68.00000,NaN,0
00mike00,38.46154,MICHAEL,NaN,NaN,US,1354590821,NaN,NaN,1.00000,NaN,0
0101craign0101,2424.08893,0101craign0101,Redmond,WA,US,1175822709,NaN,I hope to help others improve their lives. Kiv...,71.00000,NaN,0


Now we can group by country code and sum the related overall_amount_lent values

In [53]:
lend_by_country = joined.groupby('country_code')['overall_amount_lent'].sum().reset_index(name='overall_amount_lent')

In [54]:
lend_by_country.head()

,country_code,overall_amount_lent
0,AD,9824.62555
1,AE,1772123.01851
2,AF,157661.35104
3,AG,4726.33364
4,AI,919.54552


Ok, lend_by_country has been computed. The only thing that remain to do is to take back the borrow_by_country dataframe computed at point 4,5,6 and joint it with the fresh one.

In [55]:
overall_in_out = pd.concat([borrow_by_country.set_index('country_code'), lend_by_country.set_index('country_code')], axis = 1, join='inner')

Now make a simple subtraction betwee overall_amount_lent and overall_amount_borrowed columns

In [56]:
overall_in_out['lent - borrowed'] = overall_in_out['overall_amount_lent'] - overall_in_out['overall_amount_borrowed']

In [57]:
overall_in_out.sort_values('lent - borrowed', ascending=False).head(20)

,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total,overall_amount_lent,lent - borrowed
country_code,,,,,
US,9180,46352000.00000,3.92336,632648270.95044,586296270.95044
CA,1,50000.00000,0.00423,84783640.42828,84733640.42828
CN,134,380525.00000,0.03221,2423067.49042,2042542.49042
BR,482,1192325.00000,0.10092,2079209.14753,886884.14753
TH,247,608925.00000,0.05154,1271663.29514,662738.29514
VU,4,9250.00000,0.00078,296091.07369,286841.07369
UY,1,8000.00000,0.00068,104631.39063,96631.39063
VI,2,10000.00000,0.00085,20156.05897,10156.05897
GU,4,17300.00000,0.00146,26688.23040,9388.23040


As we can see in the table above, USA is the country which the greater difference between money lent and money borrowed.

### Point 10: 
Which country has the highest ratio between the difference computed at the previous point and the population?
### Point 11: 
Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

For the accomplishment of those assignments, the overall in out dataset computed before needs to be enriched with country_stats. Let's import it by selecting only the needed columns (country_code, population and population_below_poverty_line)

In [58]:
country_stats = pd.read_csv('data/country_stats.csv')[['country_code', 'population', 'population_below_poverty_line']]

Perform an inner join on country_code column

In [59]:
overall_in_out = pd.concat([overall_in_out, country_stats.set_index('country_code')], axis=1, join='inner')

In [60]:
overall_in_out.head()

,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total,overall_amount_lent,lent - borrowed,population,population_below_poverty_line
country_code,,,,,,,
AF,2337,1967950.00000,0.16657,157661.35104,-1810288.64896,35530081,35.80000
AL,3075,4307350.00000,0.36459,18904.93728,-4288445.06272,2930187,14.30000
AM,13952,22950475.00000,1.94259,30912.52234,-22919562.47766,2930450,32.00000
AZ,10172,14784625.00000,1.25141,14241.90652,-14770383.09348,9827589,4.90000
BA,608,477250.00000,0.04040,65854.69595,-411395.30405,3507017,17.20000


Calculate the ration between the lent/borrow difference and the population

In [61]:
overall_in_out['lent - borrow / pop'] = overall_in_out['lent - borrowed'] / overall_in_out['population']

Canada has the highest ratio

In [62]:
overall_in_out.sort_values('lent - borrow / pop', ascending = False).iloc[0]

borrow_count                           1.00000
overall_amount_borrowed            50000.00000
overall_amount_borrowed/total          0.00423
overall_amount_lent             84783640.42828
lent - borrowed                 84733640.42828
population                      36624199.00000
population_below_poverty_line          9.40000
lent - borrow / pop                    2.31360
Name: CA, dtype: float64

Calculate the ration between the lent/borrow difference and the population below poverty line

In [63]:
overall_in_out['lent - borrowed / pop_over_poverty_line'] = overall_in_out['lent - borrowed'] / (overall_in_out['population'] * (100 - overall_in_out['population_below_poverty_line']))

In [64]:
overall_in_out.sort_values('lent - borrowed / pop_over_poverty_line', ascending = False).iloc[0]

borrow_count                                     1.00000
overall_amount_borrowed                      50000.00000
overall_amount_borrowed/total                    0.00423
overall_amount_lent                       84783640.42828
lent - borrowed                           84733640.42828
population                                36624199.00000
population_below_poverty_line                    9.40000
lent - borrow / pop                              2.31360
lent - borrowed / pop_over_poverty_line          0.02554
Name: CA, dtype: float64

### Point 12
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018

In [65]:
loans.head()

,loan_id,country_code,loan_amount,num_lenders_total,disburse_time,planned_expiration_time,duration,base_contribution
0,657307,PH,125.00000,3,2013-12-22 08:00:00,2014-02-14 03:30:06,53 days 19:30:06,41.66667
1,657259,HN,400.00000,11,2013-12-20 08:00:00,2014-03-26 22:25:07,96 days 14:25:07,36.36364
2,658010,PK,400.00000,16,2014-01-09 08:00:00,2014-02-15 21:10:05,37 days 13:10:05,25.00000
3,659347,KG,625.00000,21,2014-01-17 08:00:00,2014-02-21 03:10:02,34 days 19:10:02,29.76190
4,656933,PH,425.00000,15,2013-12-17 08:00:00,2014-02-13 06:10:02,57 days 22:10:02,28.33333


For the accomplishment of this point, we need to remove all the loans with nan planned expiration time or disburse time.

In [66]:
loans = loans.dropna(subset=['planned_expiration_time'], how='all')
loans = loans.dropna(subset=['disburse_time'], how='all')

We're asked to equally distribute the loan amount among the loan years. To perform that, we'll make use of datetime library and we have to take into account for leap years.
Below a function which allow to do that:

In [117]:
 """
   Equally Distribute the loan amount among loan years.
    
    Params
    ------
    row : pandas.Series
        row of the loans dataframe

    Returns
    -------
    list
        Returns a list of tuples containing the pair (year, amount)
    """
def compute_amounts(row):
    amounts = []
    duration = row.duration.days
    disburse = row.disburse_time.date()
    planned = row.planned_expiration_time.date()
   
    idx = datetime.date(disburse.year, disburse.month, disburse.day) + datetime.timedelta(days=1)
    while idx <= planned:
        if idx.year == planned.year:
            try:
                amounts.append((idx.year, row.loan_amount * (planned - idx).days / duration))
            except ZeroDivisionError:
                # Duration is less than 1 day!
                amounts.append((idx.year, row.loan_amount))
            break
        else:
            amount = row.loan_amount * ((datetime.date(idx.year, 12, 31) - idx).days  + 1)/ duration
            amounts.append((idx.year, amount))
        idx = datetime.date(idx.year + 1, 1, 1)
    return amounts

In [ ]:
%%time

new_df = pd.DataFrame(columns=['loan_id', 'year', 'amount'])
ids = []
years = []
amounts = []
subset = loans[['loan_id', 'loan_amount', 'disburse_time', 'planned_expiration_time', 'duration']]
for _, row in subset.iterrows():
    amts = compute_amounts(row)
    for year, amount in amts:
        ids.append(row.loan_id)
        years.append(year)
        amounts.append(amount)
        
new_df['loan_id'] = ids
new_df['year'] = years
new_df['amount'] = amounts

To demonstrate that this function works, we can chose a loan as example (in our case, the one with loan id = 1077955) and see if the loan amount is equally distributed among the years.

In [112]:
example = new_df[new_df.loan_id == 1077955]

In [113]:
example

,loan_id,year,amount
0,1077955,2011,24.25713
1,1077955,2012,1109.76349
2,1077955,2013,1106.73135
3,1077955,2014,1106.73135
4,1077955,2015,1106.73135
5,1077955,2016,545.78532


In [114]:
subset[subset.loan_id == 1077955]

,loan_id,loan_amount,disburse_time,planned_expiration_time,duration
567435,1077955,5000.00000,2011-12-23 20:24:04,2016-06-29 01:46:39,1649 days 05:22:35


In [115]:
sum(example.amount)

5000.0